# DeFi valuation Example

## Uniswap v2 and clones 


This is notebook demos the potential valuation methodology for LP position in Uniswap V2 DEX. 

In [10]:
import json
from web3 import Web3
import requests
from time import sleep
from datetime import datetime

#we are using FTX as price oracle - in the example I am making FTX is one of the most liquid markets (and we are assuming deposits are available)
def last_price(symbol):
    if symbol == 'WETH':
        symbol = 'ETH'
    res = requests.get(f'https://ftx.us/api/markets/{symbol}/USD').json()['result']['last']
    return res

#node provider - this a demo provider, I would suggest Point72 to deploy a node (so all infra are internal - no security breaches and point of failures)
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/d4fff350e2ad4993b8e067b25cb6f171'))

#contracts ABI - to direclty interact with contracts 
with open('lp_abi.json') as json_file:
    abi_lp = json.load(json_file)
with open('erc_20_abi.json') as json_file:
    erc_20_abi = json.load(json_file)
    

In [15]:
class pool:
    def __init__(self, address):
        self.address = address
        self.pool_contr = w3.eth.contract(address, abi=abi_lp)

        #fetch pool tokens addresses (eg. usdt:0xdAC17F958D2ee523a2206206994597C13D831ec7)
        self.token0 = self.pool_contr.functions.token0().call()
        self.token1 = self.pool_contr.functions.token1().call()

        #create contract object 
        self.token0_contr = w3.eth.contract(self.token0, abi=erc_20_abi)
        self.token1_contr = w3.eth.contract(self.token1, abi=erc_20_abi)

        #fect tokens decimals
        self.token0_decimals = self.token0_contr.functions.decimals().call()
        self.token1_decimals = self.token1_contr.functions.decimals().call()

        #fetch tokens symbols
        self.token0_symbol = self.token0_contr.functions.symbol().call()
        self.token1_symbol = self.token1_contr.functions.symbol().call()
        
        print(f'token0: {self.token0_symbol} - {self.token0}')
        print(f'token1: {self.token1_symbol} - {self.token1}')
        
    def update(self, now):
        #fetch LP total supply - normalize for decimals
        self.total_supply = self.pool_contr.functions.totalSupply().call()/10**18

        #calculate tokens liquidity - DEXes call it reserves
        self.token0_reserves = self.pool_contr.functions.getReserves().call()[0]/10**self.token0_decimals
        self.token1_reserves = self.pool_contr.functions.getReserves().call()[1]/10**self.token1_decimals

        #calculate tokens last price - using FTX spot as oracle
        self.token0_price = last_price(self.token0_symbol)
        self.token1_price = last_price(self.token1_symbol)

        #calculate pool market cap
        self.pool_marketcap = (self.token0_reserves*self.token0_price) + (self.token1_reserves*self.token1_price)

        #calculate LP value
        self.lp_value = self.pool_marketcap/self.total_supply

        print('--------------')
        print(f'{now} - pool_market_cap: {self.pool_marketcap: .2f},  LP supply: {self.total_supply: .2f}')
        print(f'{now} - Total LP supply is {self.total_supply: .2f}, pool market cap is {self.pool_marketcap: .2f}, dollar value of 0.01 LP is {self.lp_value/100: .2f}')
        
        return self.lp_value

In [19]:
#contract addresses - everything (tokens, symbols) can be retrieved just by pool contract - this makes the methodology highly scalable:
eth_usdt = '0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852'

pool_inst = pool(eth_usdt)

#update every 5 seconds LP token value
while True:
    now = datetime.now().strftime("%H:%M:%S")

    pool_inst.update(now)
    sleep(5)



token0: WETH - 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
token1: USDT - 0xdAC17F958D2ee523a2206206994597C13D831ec7
--------------
18:36:19 - pool_market_cap:  34026538.07,  LP supply:  0.20
18:36:19 - Total LP supply is  0.20, pool market cap is  34026538.07, dollar value of 0.01 LP is  1725850.70
--------------
18:36:25 - pool_market_cap:  34026546.30,  LP supply:  0.20
18:36:25 - Total LP supply is  0.20, pool market cap is  34026546.30, dollar value of 0.01 LP is  1725851.12
--------------
18:36:32 - pool_market_cap:  34028642.01,  LP supply:  0.20
18:36:32 - Total LP supply is  0.20, pool market cap is  34028642.01, dollar value of 0.01 LP is  1725957.42
--------------
18:36:38 - pool_market_cap:  34025498.44,  LP supply:  0.20
18:36:38 - Total LP supply is  0.20, pool market cap is  34025498.44, dollar value of 0.01 LP is  1725797.97


KeyboardInterrupt: 